$\large{\textbf{Solution to Question 2.}}$

In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.6 MB 8.8 MB/s 
     |████████████████████████████████| 49 kB 6.7 MB/s 


In [2]:
from pyomo.environ import *

In [3]:
import numpy as np

In [5]:
# create a model
model = ConcreteModel()

In [6]:
N = 12 # No. of variables
M = 5 # No. of constraints

In [7]:
#Objective function coefficients
obj_coef_c = np.array([0.56, 1.721, -9.6, -2.85, 41.5, 33.05, -0.75, 20.5, -0.35, -31.02, 50.65, 6.125]) 

In [10]:
#Defining the constraints matrix
constr_coef_A = np.array([[1, -1, 0, 0, 2, -2, 1, 0, 0, -1, 1, -2], 
              [1, 0, 0, -1, 0, -2, 0, -1, 1, 0, 0, -1],
              [-1, 0, 0, 1, -1, 2.5, -3.5, 0, 2, -1, 1, 0],
              [0, 1, -0.45, 0, -2.78, 0, -3.4, 0, -1, 1, -1, 0.25],
              [0, -1, 1, 0, 1, 0, 1, 3, -2, 0, 1, 0]])

In [11]:
#Creating the array for RHS of constraints
constr_rhs_b = np.array([88.5, 35.4, 47.9, 41.43, 22.7]) 

In [13]:
#Setting the lower and upper bounds
lower_bound = np.array([0, 0, -np.inf, -np.inf, -34, 0, -67, 0, -23, -5, 6, 1])
upper_bound = np.array([12, 503, 4, np.inf, np.inf, 2, 20, 55, 23, 15, np.inf, np.inf])

In [14]:
col_indices = np.arange(N)
row_indices = np.arange(M)

In [15]:
#declare the decision variables in the model
model.x = Var(col_indices)

In [16]:
# create a ConstraintList to hold multiple constraints
model.constraints = ConstraintList()

In [18]:
#Adding the constraints to the constraint list
for i in row_indices:
  model.constraints.add(sum(constr_coef_A[i][j]*model.x[j] for j in col_indices) <= constr_rhs_b[i])

In [19]:
for j in col_indices:
  model.x[j].setlb(lower_bound[j])
  model.x[j].setub(upper_bound[j])

In [20]:
# add the model objective
model.objective = Objective(expr = sum(obj_coef_c[j]*model.x[j] for j in col_indices), sense=minimize)

In [21]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    x : Size=12, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :   0.0 :  None :  12.0 : False :  True :  Reals
          1 :   0.0 :  None : 503.0 : False :  True :  Reals
          2 :  None :  None :   4.0 : False :  True :  Reals
          3 :  None :  None :  None : False :  True :  Reals
          4 : -34.0 :  None :  None : False :  True :  Reals
          5 :   0.0 :  None :   2.0 : False :  True :  Reals
          6 : -67.0 :  None :  20.0 : False :  True :  Reals
          7 :   0.0 :  None :  55.0 : False :  True :  Reals
          8 : -23.0 :  None :  23.0 : False 

In [22]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155680 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [23]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -1951.831
  Upper bound: -1951.831
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 13
  Number of nonzeros: 38
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.01206207275390625
# ----------------------------------------------------------
#   Solution Information
# ---------------------------

In [24]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()


Objective =  -1951.8309999999997

Decision Variables
x[ 0 ] =  12.0
x[ 1 ] =  0.0
x[ 2 ] =  4.0
x[ 3 ] =  119.82
x[ 4 ] =  -34.0
x[ 5 ] =  0.0
x[ 6 ] =  20.0
x[ 7 ] =  0.0
x[ 8 ] =  -7.46000000000002
x[ 9 ] =  15.0
x[ 10 ] =  6.0
x[ 11 ] =  1.0

Constraints
constraints : Size=5
    Key : Lower : Body                : Upper
      1 :  None :               -47.0 :  88.5
      2 :  None : -116.28000000000002 :  35.4
      3 :  None :   47.89999999999995 :  47.9
      4 :  None :   41.43000000000002 : 41.43
      5 :  None :   10.92000000000004 :  22.7


$\large{\textbf{Answers:}}$

$\textbf{Objective Function Value = }-1951.831 \\ \textbf{Value of decision variables:} \\ x_1 = 12 \\ x_2 = 0 \\ x_3 = 4 \\ x_4 = 119.82 \\ x_5 = -34 \\ x_6 = 0 \\ x_7 = 20 \\ x_8 = 0 \\ x_9 = -7.46 \\ x_{10} = 15 \\ x_{11} = 6 \\ x_{12} = 1$

$\textbf{2.2 Activity of constraints:}\\ \textbf{Constraint 1: } \text{It is inactive as its value is -47 which is strictly less than the RHS = 88.5.}\\ \textbf{Constraint 2: } \text{It is inactive as its value is -116.28 which is strictly less than the RHS = 35.4.} \\ \textbf{Constraint 3: } \text{It is active as its value is 47.9 which is exactly equal to the RHS.} \\ \textbf{Constraint 4: } \text{It is active as its value is 41.43 which is exactly equal to the RHS.} \\ \textbf{Constraint 1: } \text{It is inactive as its value is 10.92 which is strictly less than the RHS = 22.7.}$